In [5]:
import nibabel as nib
import matplotlib.pyplot as plt

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import numpy as np
import torchvision.models as models
import torch.nn.functional as F

In [10]:
# 自定义数据集类
class SpineWeb15(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        
    def __len__(self):
        return len(self.data_path)
    
    def __getitem__(self, index):
        metadata = np.load(self.data_path+str(index)+".npy")
        image = metadata[0]
        label = metadata[1]

        image_data = torch.from_numpy(image).float()
        label_data = torch.from_numpy(label).float()
        
        if self.transform:
            image_data = self.transform(image_data)
        return image_data, label_data



In [18]:
# U-Net++ structure

class UnetEncoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UnetEncoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        skip_connection = x
        x = self.pool(x)
        return x, skip_connection

# 定义Unet++的Decoder部分
class UnetDecoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UnetDecoder, self).__init__()
        self.upconv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x, skip_connection):
        x = self.upconv(x)
        # skip_connection = F.interpolate(skip_connection, size=x.size()[2:], mode='bilinear')
        x = torch.cat((x, skip_connection), dim=1)  # x.dimension + skip_connection = in_channels.dimension
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return x

# 定义Unet++模型
class UnetPlusPlus(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UnetPlusPlus, self).__init__()
        self.encoder1 = UnetEncoder(in_channels, 64)
        self.encoder2 = UnetEncoder(64, 128)
        self.encoder3 = UnetEncoder(128, 256)
        self.encoder4 = UnetEncoder(256, 512)
        
        self.middleconv1 = nn.Conv2d(512, 1024, kernel_size=3, padding=1)
        self.middleconv2 = nn.Conv2d(1024, 1024, kernel_size=3, padding=1)
        
        self.decoder4 = UnetDecoder(1024, 512)
        self.decoder3 = UnetDecoder(512, 256)
        self.decoder2 = UnetDecoder(256, 128)
        self.decoder1 = UnetDecoder(128, 64)
        
        self.output = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        x, skip_connection1 = self.encoder1(x)
        x, skip_connection2 = self.encoder2(x)
        x, skip_connection3 = self.encoder3(x)
        x, skip_connection4 = self.encoder4(x)  # 最后一层Encoder不需要skip connection

        x = F.relu(self.middleconv1(x))
        x = F.relu(self.middleconv2(x))
        
        x = self.decoder4(x, skip_connection4)  #x:1024->512, skip_connection4:512
        x = self.decoder3(x, skip_connection3)  #x:512->256, skip_connection3:256
        x = self.decoder2(x, skip_connection2)  #x:256->128, skip_connection2:128
        x = self.decoder1(x, skip_connection1)  #x:128->64, skip_connection1:64
        
        x = self.output(x)
        x = torch.sigmoid(x)  # 使用sigmoid激活函数输出分割结果
        
        return x


In [20]:
# -----------------------------
# UNet++ All Set
# -----------------------------

# empty the cache for model and training process
torch.cuda.empty_cache()

# set the hyper parameters and the paths 设置超参数和路径
data_path = "./dataset/Spineweb_dataset15/processed/"

#hyper parameters set
batch_size = 32
num_epochs = 10
learning_rate = 0.0001
confidence = 0.5
in_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数
out_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数

# 创建数据集和数据加载器
transform = transforms.Compose([
    transforms.ToPILImage(),  # 转换为 PIL 图像对象
    transforms.Resize((128, 128)),  # 调整大小为 128x128
    transforms.ToTensor()  # 转换为张量
])
dataset = SpineWeb15(data_path=data_path,transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 创建模型和优化器
model = UnetPlusPlus(in_channels=in_channels, out_channels=out_channels)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()  # 二分类任务可以使用BCEWithLogitsLoss

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# check the parameters of the model could be trained 检查模型的参数是否设置为可训练
# for name, param in model.named_parameters():
#     print(f"Parameter: {name}, Requires Grad: {param.requires_grad}")

best_loss = 100.0
best_IoU = 0.0
best_Dice = 0.0

# 训练模型
for epoch in range(num_epochs):
    torch.cuda.empty_cache()
    model.train()
    train_loss = 0
    total_iou = 0
    total_dice = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        # squeeze the dimension that pytorch add for channels
        outputs = outputs.squeeze(1)     # tensor size ([16,128,128])
        # process the result, bigger than 0.5 is reliable
        predicted = (outputs > confidence).float()     # tensor size ([16,128,128])

        intersection = torch.logical_and(predicted, labels).sum((1, 2))
        union = torch.logical_or(predicted, labels).sum((1, 2))
        iou = (intersection / (union + 1e-7)).mean()  # 平均每个样本的IoU
        dice = (2 * intersection / (predicted.sum((1, 2)) + labels.sum((1, 2)) + 1e-7)).mean()  # 平均每个样本的Dice Coefficient

        loss = criterion(outputs, labels)

        train_loss += loss.item()
        total_iou += iou.item()
        total_dice += dice.item()

        loss.backward()
        optimizer.step()

    avg_loss = train_loss / len(dataloader)
    avg_iou = total_iou / len(dataloader)
    avg_dice = total_dice / len(dataloader)
    
    # get the best IoU and Dice
    best_loss = best_loss if best_loss < avg_loss else avg_loss
    best_IoU = best_IoU if best_IoU > avg_iou else avg_iou
    best_Dice = best_Dice if best_Dice > avg_dice else avg_dice

    print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}, Avg IoU: {avg_iou:.4f}, Avg Dice: {avg_dice:.4f}")
print("-----------------------------------------")
print(f"Best Loss: {best_loss:.4f}, Best IoU: {best_IoU:.4f}, Best Dice: {best_Dice:.4f}")
# empty the cache for model and training process
torch.cuda.empty_cache()

KeyboardInterrupt: 

In [6]:
# -----------------------------
# UNet++ Train and Test Set
# -----------------------------

def calculate_iou(prediction, target):
    intersection = torch.logical_and(prediction, target).sum()
    union = torch.logical_or(prediction, target).sum()
    iou = intersection / union
    return iou.item()

def calculate_dice(prediction, target):
    intersection = torch.logical_and(prediction, target).sum()
    dice = (2 * intersection) / (prediction.sum() + target.sum())
    return dice.item()

# empty the cache for model and training process
torch.cuda.empty_cache()

# set the hyper parameters and the paths 设置超参数和路径
data_path = "./dataset/Spineweb_dataset15/train/"
test_path = "./dataset/Spineweb_dataset15/test/"

#hyper parameters set
batch_size = 32
num_epochs = 10
learning_rate = 0.0001
confidence = 0.5
in_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数
out_channels = 1  # according to demand to adjust the number of channels 根据实际情况修改通道数

# 创建数据集和数据加载器
transform = transforms.Compose([
    transforms.ToPILImage(),  # 转换为 PIL 图像对象
    transforms.Resize((128, 128)),  # 调整大小为 128x128
    transforms.ToTensor()  # 转换为张量
])
dataset = SpineWeb15(data_path=data_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
testset = SpineWeb15(data_path=test_path, transform=transform)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True)

# 创建模型和优化器
model = UnetPlusPlus(in_channels, out_channels)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()  # 二分类任务可以使用BCEWithLogitsLoss

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# check the parameters of the model could be trained 检查模型的参数是否设置为可训练
# for name, param in model.named_parameters():
#     print(f"Parameter: {name}, Requires Grad: {param.requires_grad}")

best_loss_train = 100.0
best_IoU_train = 0.0
best_Dice_train = 0.0

best_IoU_test = 0.0
best_Dice_test = 0.0

# 训练模型
for epoch in range(num_epochs):
    # torch.cuda.empty_cache()
    model.train()
    train_loss = 0
    total_iou = 0
    total_dice = 0
    total_iou_test = 0
    total_dice_test = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
    
        optimizer.zero_grad()

        outputs = model(images)
        # squeeze the dimension that pytorch add for channels
        outputs = outputs.squeeze(1)     # tensor size ([16,128,128])
        # process the result, bigger than 0.5 is reliable
        predicted = (outputs > confidence).float()     # tensor size ([16,128,128])

        intersection = torch.logical_and(predicted, labels).sum((1, 2))
        union = torch.logical_or(predicted, labels).sum((1, 2))
        iou = (intersection / (union + 1e-7)).mean()  # 平均每个样本的IoU
        dice = (2 * intersection / (predicted.sum((1, 2)) + labels.sum((1, 2)) + 1e-7)).mean()  # 平均每个样本的Dice Coefficient

        loss = criterion(outputs, labels)

        train_loss += loss.item()
        total_iou += iou.item()
        total_dice += dice.item()

        loss.backward()
        optimizer.step()

    # train set results
    avg_loss = train_loss / len(dataloader)
    avg_iou = total_iou / len(dataloader)
    avg_dice = total_dice / len(dataloader)

    # test set results
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            # squeeze the dimension that pytorch add for channels
            outputs = outputs.squeeze(1)     # tensor size ([16,128,128])
            # process the result, bigger than 0.5 is reliable
            predicted = (outputs > confidence).float()     # tensor size ([16,128,128])
            
            intersection = torch.logical_and(predicted, targets).sum((1, 2))
            union = torch.logical_or(predicted, targets).sum((1, 2))
            iou = (intersection / (union + 1e-7)).mean()  # 平均每个样本的IoU
            dice = (2 * intersection / (predicted.sum((1, 2)) + targets.sum((1, 2)) + 1e-7)).mean()  # 平均每个样本的Dice Coefficient

            total_iou_test += iou.item()
            total_dice_test += dice.item()

    test_iou = total_iou_test / len(test_loader)
    test_dice = total_dice_test / len(test_loader)

    # get the best IoU and Dice
    best_loss_train = best_loss_train if best_loss_train < avg_loss else avg_loss
    best_IoU_train = best_IoU_train if best_IoU_train > avg_iou else avg_iou
    best_Dice_train = best_Dice_train if best_Dice_train > avg_dice else avg_dice
    best_IoU_test = best_IoU_test if best_IoU_test > test_iou else test_iou
    # best_Dice_test = best_Dice_test if best_Dice_test > test_dice else test_dice

    if(best_Dice_test <= test_dice):
        best_Dice_test = test_dice
        torch.save(model.state_dict(), f"./model/UNetPP_checkpoint.pth")

    print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}, Avg IoU: {avg_iou:.4f}, Avg Dice: {avg_dice:.4f},test IoU: {test_iou:.4f}, test Dice: {test_dice:.4f}")
    
print("-----------------------------------------")
print(f"Best Loss: {best_loss_train:.4f}, Best Train IoU: {best_IoU_train:.4f}, Best Train Dice: {best_Dice_train:.4f}, Best Test IoU: {best_IoU_test:.4f}, Best Test Dice: {best_Dice_test:.4f}")
# empty the cache for model and training process
torch.cuda.empty_cache()

NameError: name 'test_dataloader' is not defined